In [ ]:
import pandas as pd
from pandas import Timestamp
import datetime as dt
import os
import yfinance as yf
import ccxt


In [ ]:
def indexConverter(data):
    data['index'] = data.index
    data['index'] = data['index'].apply(lambda x: Timestamp(x))
    data['index'] = data['index'].apply(
        lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
    data['index'] = pd.to_datetime(data['index'])
    data = data.set_index('index')
    data = data.sort_index()
    return data


In [ ]:
def binance(ticket, period: int = 1000, interval='1w', params: any = {}):
    binance = ccxt.binance()
    binance.fetch_ticker(ticket)
    data = binance.fetch_ohlcv(ticket, interval, limit=period, params=params)
    data = pd.DataFrame(
        data, columns=['Date', 'Open', 'High', 'Low', 'Close', 'Volume'])
    data['Date'] = pd.to_datetime(data['Date'], unit='ms')
    data = data.set_index('Date')
    data = data.sort_index()
    return data[-period:]


In [ ]:
def yahooFinance(baseAsset, end: str, quoteAsset: str = "USDT",  interval='1w'):
    try:
        if quoteAsset.upper().__contains__("USDT"):
            quoteAsset = "USD"
        ticket = baseAsset+"-"+quoteAsset
        if interval == '1w':
            interval = '1wk'
        res = yf.download(ticket, period='max',
                          interval=interval, end=end)
        return res
    except Exception as e:
        print(e)
        return None


In [ ]:
def verifyData(lastDataIndexSaved, interval):
    now = dt.datetime.now()
    if interval == '1w':
        interval = 7
    if interval == '1d':
        interval = 1
    return now - lastDataIndexSaved > dt.timedelta(days=interval)   
    

In [ ]:
def getData(baseAsset, quoteAsset="USDT", period=1000, interval='1w'):
    baseAsset = baseAsset.upper()
    pathName = "data/"+baseAsset.lower()+"-"+quoteAsset.lower()+"_"+interval+".csv"
    if not os.path.exists(pathName):
        print("Baixando dados da Binance")
        data = binance(baseAsset+"/"+quoteAsset, period, interval)
        data = indexConverter(data)
        if period > len(data):
            print("Baixando dados do Yahoo Finance")
            res = yahooFinance(baseAsset, data.index[-1], quoteAsset, interval)
            res = indexConverter(res)
            data = data.append(res)
            data = data.reset_index().drop_duplicates(
                subset='index', keep='first').set_index('index')
            data = data.sort_index()
    else:
        print("Carregando dados do arquivo: "+pathName)
        data = pd.read_csv(pathName, index_col=0)
        data = indexConverter(data)
        if verifyData(data.index[-1], interval):
            dataUpdate = binance(baseAsset+"/"+quoteAsset, period, interval)
            dataUpdate = indexConverter(dataUpdate)
            data = data.append(dataUpdate)
            data = data.reset_index().drop_duplicates(
                subset='index', keep='first').set_index('index')
            data = data.sort_index()
    data.to_csv(pathName)
    return data[-period:]


In [ ]:
import sys
import os
import backtesting 
from backtesting import Backtest
import pandas as pd
import datetime as dt
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: sys.path.append(dir1)
from src.strategys.HiLo import HiLoStrategy as HiLoStrategy
import pandas as pd
backtesting.set_bokeh_output(notebook=False)
data = getData("BTC")
HiLoStrategy.setInitialParams(HiLoStrategy, [3,2])
bt = Backtest(data, HiLoStrategy, cash=1000000000, exclusive_orders=False)
output = bt.run()
print(output)
bt.plot()